# Estimación de dirección de arribo de fuentes sonoras 

In [1]:
import numpy as np
import pandas as pd
import gspread # Extension para leer el excel desde Drive
from oauth2client.service_account import ServiceAccountCredentials # Extensión para autentificación con Drive
from tdoa_py import correlation, simulation_room

In [2]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=0#gid=0")
sheet = spreadsheet.sheet1

# Leer datos
data = sheet.get_all_records()
df_simulations = pd.DataFrame(data)

df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla luego
df_simulations.head() # Verificar el resultado

,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,src_az_deg,src_z,theta_1,theta_2,theta_3,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim
0,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,0,1.2,,,,,,,,"(100, 100, 100)"
1,2,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,10,1.2,,,,,,,,"(100, 100, 100)"
2,3,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,20,1.2,,,,,,,,"(100, 100, 100)"
3,4,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,30,1.2,,,,,,,,"(100, 100, 100)"
4,5,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,40,1.2,,,,,,,,"(100, 100, 100)"


In [3]:
# Iterar sobre el DataFrame con indexación correcta
for idx, sim in df_simulations.iterrows():
    src_az_deg, mic_signals = simulation_room.sim_room_Nmics(
        wav_path=f'audios/anechoic/{sim["audio"]}',
        out_dir=f'audios/output/{sim["sim_name"]}',
        fs=48000,
        room_dim=sim["room_dim"],
        rt60=sim["rt60"],
        snr_db=sim["snr_db"],
        n_mics=sim["n_mics"],
        mic_d=sim["mic_d"],
        mic_z=sim["mic_z"],
        mic_directivity=sim["mic_directivity"],
        src_dist=sim["src_dist"],
        src_az_deg=sim["src_az_deg"],
        src_z=sim["src_z"],
        save_audio=False
    )
    
    avg_angle_deg, avg_tdoa, hemi_avgs, tdoas, angles = correlation.estimate_doa(
        signal_input=mic_signals,
        mic_d=sim["mic_d"],
        method=sim["method"],
        fs=48000
    )

    # Guardar resultados en cada fila individual
    df_simulations.at[idx, 'avg_angle_deg'] = avg_angle_deg
    df_simulations.at[idx, 'avg_tdoa'] = avg_tdoa
    df_simulations.at[idx, 'hemi_1'] = hemi_avgs['H1']
    df_simulations.at[idx, 'hemi_2'] = hemi_avgs['H2']
    df_simulations.at[idx, 'theta_1'] = angles[0]
    df_simulations.at[idx, 'theta_2'] = angles[1]
    df_simulations.at[idx, 'theta_3'] = angles[2]

# Convertir el DataFrame a una lista de listas para Google Sheets
df_simulations["room_dim"] = df_simulations["room_dim"].apply(lambda x: ",".join(map(str, x)))
data = [df_simulations.columns.values.tolist()] + df_simulations.values.tolist()

# Escribir en la hoja de cálculo
sheet.update(data)
print("DataFrame guardado correctamente en Google Sheets.")

df_simulations



[[49.85 49.95 50.05 50.15]
 [50.   50.   50.   50.  ]
 [ 1.2   1.2   1.2   1.2 ]]
[np.float64(70.0), np.float64(50.0), 1.2]
4
2
t_0_2 =  0.0005833333333333334
d_0_2 0.2
cos_val 1.0
t_1_2 =  0.0002916666666666667
d_1_2 0.1
cos_val 1.0
t_2_2 =  0.0
d_2_2 0.0
t_3_2 =  0.0002916666666666667
d_3_2 0.1
cos_val 1.0
[[49.85 49.95 50.05 50.15]
 [50.   50.   50.   50.  ]
 [ 1.2   1.2   1.2   1.2 ]]
[np.float64(69.69615506024417), np.float64(53.47296355333861), 1.2]
4
2
t_0_2 =  0.0005833333333333334
d_0_2 0.2
cos_val 1.0
t_1_2 =  0.0002916666666666667
d_1_2 0.1
cos_val 1.0
t_2_2 =  0.0
d_2_2 0.0
t_3_2 =  0.0002916666666666667
d_3_2 0.1
cos_val 1.0
[[49.85 49.95 50.05 50.15]
 [50.   50.   50.   50.  ]
 [ 1.2   1.2   1.2   1.2 ]]
[np.float64(68.79385241571816), np.float64(56.84040286651337), 1.2]
4
2
t_0_2 =  0.0005416666666666666
d_0_2 0.2
cos_val 0.9289583333333332
t_1_2 =  0.0002708333333333333
d_1_2 0.1
cos_val 0.9289583333333332
t_2_2 =  0.0
d_2_2 0.0
t_3_2 =  0.0002708333333333333
d_3_2 0.1


,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,src_az_deg,src_z,theta_1,theta_2,theta_3,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim
0,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,0,1.2,0.0,0.0,0.0,0.0,0.000292,0.0,0.0,"100,100,100"
1,2,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,10,1.2,0.0,0.0,0.0,0.0,0.000292,0.0,0.0,"100,100,100"
2,3,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,20,1.2,21.726984,21.726984,21.726984,21.726984,0.000271,21.726984,21.726984,"100,100,100"
3,4,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,30,1.2,30.962969,30.962969,30.962969,30.962969,0.00025,30.962969,30.962969,"100,100,100"
4,5,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,40,1.2,41.382545,38.182877,38.182877,39.249433,0.000224,39.249433,39.249433,"100,100,100"
5,6,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,50,1.2,49.97476,49.97476,49.97476,49.97476,0.000188,49.97476,49.97476,"100,100,100"
6,7,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,60,1.2,59.986216,59.986216,59.986216,59.986216,0.000146,59.986216,59.986216,"100,100,100"
7,8,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,70,1.2,69.066039,69.066039,69.066039,69.066039,0.000104,69.066039,69.066039,"100,100,100"
8,9,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,80,1.2,79.709107,81.783343,81.783343,81.091931,0.000047,81.091931,81.091931,"100,100,100"
9,10,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,90,1.2,90.0,90.0,90.0,90.0,0.0,90.0,90.0,"100,100,100"
